In [3]:
import plotly.plotly as py
import plotly.graph_objs as go
import plotly
import pyodbc
import pandas as pd
import plotly.plotly as py
import cufflinks as cf
import pandas as pd
import numpy as np
import plotly.graph_objs as go


In [10]:

def database_connection():
    conn = pyodbc.connect(
        r'DRIVER={SQL Server Native Client 11.0};'
        r'SERVER=localhost;'
        r'DATABASE=seis603_pricing_project;'
        r'UID=python_dev;'
        r'PWD=python_dev;'
        )
    cursor = conn.cursor()
    return cursor

db_connection = database_connection()

In [11]:
#items tracking
sql = "EXEC [dbo].[usp_GetCurrentTracking] {}".format(user_id)
db_connection.execute(sql)#, params) #executing sproc
list_items = db_connection.fetchall()#[0] #fetchone will only return first result
#df = pd.Series(list_items)
labels = ['item id','item name']
df = pd.DataFrame.from_records(list_items, columns=labels) #create dataframe from list
print(df)    

   item id             item name
0      101     Adrenaline GTS 17
1      102  Radian R100 Car Seat


In [12]:
#set up user and item we are looking up
user_id = 100
item_id = 102

In [25]:
#price history of items
sql = "EXEC [dbo].[usp_GetUserItemPriceHistory] {},{}".format(user_id,item_id)
db_connection.execute(sql)#, params) #executing sproc
list_items = db_connection.fetchall()#[0] #fetchone will only return first result
labels = ['item id','item name','Brand','Store ID', 'Store Name','Item Price','Price Date']
price_history_df = pd.DataFrame.from_records(list_items, columns=labels) #create dataframe from list

#pivot data
pvt_price_history =  price_history_df.pivot(index='Price Date',columns='Store Name',values = 'Item Price' ).fillna(0)

#unpvit the price 
upvt_price_history = pvt_price_history.unstack().reset_index(name='Item Price')

#replace 0 with None so we can front fill later
upvt_price_history['Item Price'].replace(
    to_replace=[0],
    value=None,
    inplace=True
)

#front fill all the data
upvt_price_history.groupby('Price Date')['Item Price'].fillna(method='bfill') 

# repivot for graphing
pvt_price_history2 =  upvt_price_history.pivot(index='Price Date',columns='Store Name',values = 'Item Price' ).fillna(0)


In [37]:
#graph data
item_name = price_history_df["item name"].unique()
dates = upvt_price_history["Price Date"].unique()
print(dates)

#store data
target = upvt_price_history.loc[upvt_price_history['Store Name'] == 'Target']
target_date = target["Price Date"]
target_price= target["Item Price"]

amazon = upvt_price_history.loc[upvt_price_history['Store Name'] == 'Amazon']
amazon_date = target["Price Date"]
amazon_price= target["Item Price"]

walmart = upvt_price_history.loc[upvt_price_history['Store Name'] == 'Walmart']
walmart_date = target["Price Date"]
walmart_price= target["Item Price"]

toysrus = upvt_price_history.loc[upvt_price_history['Store Name'] == 'Toys r Us']
toysrus_date = target["Price Date"]
toysrus_price= target["Item Price"]


#settings for graph
title = "Price History for {}".format(item_name)
labels = [store_name]
colors = ['rgb(255, 153, 0)', 'rgb(49, 86, 219)', 'rgb(205, 12, 24)', 'rgb(255, 153, 0)']
mode_size = [8,8,8,8]
line_size = [2,2,2,2]

x_data = [target_date,amazon_date,walmart_date,toysrus_date]
y_data = [target_price,amazon_price,walmart_price,toysrus_price]

print(list(x_data))
print(list(y_data))

#making empty trace
traces = []
#create trace for each store item
for i in range(0, 4):
    traces.append(go.Scatter(
        x=x_data[i],
        y=y_data[i],
        mode='lines',
        line=dict(color=colors[i], width=line_size[i]),
        connectgaps=True,
    ))

    traces.append(go.Scatter(
        x=[x_data[i][0], x_data[i][0]],
        y=[y_data[i][0], y_data[i][0]],
        mode='markers',
        marker=dict(color=colors[i], size=mode_size[i])
    ))

layout = go.Layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        autotick=False,
        ticks='outside',
        tickcolor='rgb(204, 204, 204)',
        tickwidth=2,
        ticklen=5,
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showgrid=False,
        zeroline=False,
        showline=False,
        showticklabels=False,
    ),
    autosize=False,
    margin=dict(
        autoexpand=False,
        l=100,
        r=20,
        t=110,
    ),
    showlegend=True,
)    
    
annotations = [] 
# Adding labels
for y_trace, label, color in zip(y_data, labels, colors):
    # labeling the left_side of the plot
    annotations.append(dict(xref='paper', x=0.05, y=y_trace[0],
                                  xanchor='right', yanchor='middle',
                                  text=label + ' {}%'.format(y_trace[0]),
                                  font=dict(family='Arial',
                                            size=16,
                                            color=colors,),
                                  showarrow=False))
    # labeling the right_side of the plot
    annotations.append(dict(xref='paper', x=0.95, y=y_trace[11],
                                  xanchor='left', yanchor='middle',
                                  text='{}%'.format(y_trace[11]),
                                  font=dict(family='Arial',
                                            size=16,
                                            color=colors,),
                                  showarrow=False))
# Title
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text=title,
                              font=dict(family='Arial',
                                        size=30,
                                        color='rgb(37,37,37)'),
                              showarrow=False))

layout['annotations'] = annotations

fig = go.Figure(data=traces, layout=layout)
py.iplot(fig, filename='news-source')

[datetime.date(2018, 3, 8) datetime.date(2018, 3, 9)
 datetime.date(2018, 3, 12) datetime.date(2018, 3, 13)
 datetime.date(2018, 3, 14) datetime.date(2018, 3, 15)
 datetime.date(2018, 3, 17) datetime.date(2018, 3, 18)
 datetime.date(2018, 3, 19)]
[9     2018-03-08
10    2018-03-09
11    2018-03-12
12    2018-03-13
13    2018-03-14
14    2018-03-15
15    2018-03-17
16    2018-03-18
17    2018-03-19
Name: Price Date, dtype: object, 9     2018-03-08
10    2018-03-09
11    2018-03-12
12    2018-03-13
13    2018-03-14
14    2018-03-15
15    2018-03-17
16    2018-03-18
17    2018-03-19
Name: Price Date, dtype: object, 9     2018-03-08
10    2018-03-09
11    2018-03-12
12    2018-03-13
13    2018-03-14
14    2018-03-15
15    2018-03-17
16    2018-03-18
17    2018-03-19
Name: Price Date, dtype: object, 9     2018-03-08
10    2018-03-09
11    2018-03-12
12    2018-03-13
13    2018-03-14
14    2018-03-15
15    2018-03-17
16    2018-03-18
17    2018-03-19
Name: Price Date, dtype: object]
[9     2

KeyError: 0